In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from preprocess.pre_process import multi_rle_encode, rle_encode, rle_decode, masks_as_image, masks_as_color, balancing_train
from preprocess.pre_process import make_image_gen, create_aug_gen

import time
from IPython.display import clear_output

Using TensorFlow backend.


In [2]:
ship_dir = '../../data/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir, 'train')# Images for training
test_image_dir = os.path.join(ship_dir, 'test')# Images for testing
label_dir = os.path.join(ship_dir, 'train_ship_segmentations_v2.csv')# Images for testing
masks = pd.read_csv(label_dir, engine="python") # Markers for ships

In [3]:
print(len(masks.ImageId))
print(len(masks.EncodedPixels))

231723
231723


## Remove not found images

In [ ]:
# num_examples = len(masks.ImageId)
t_per_file = []
start = time.time()
list_error_index = []
for index, imgId in enumerate(masks.ImageId) :
    if index%100==0 or index==num_examples-1:
        clear_output(wait=True)
        t_per_file.append((time.time() - start) / 100)
        eta = np.mean(t_per_file) * (num_examples - index)
        print("ETA:", round(eta/60,2), end=" minutes ==>  ")
        print(index, "/", num_examples, end" : ")
        print(len(list_error_index), "errors detected")
        start = time.time()
    img_path = os.path.join(train_image_dir, imgId)
    try:
        img = imread(img_path)
    except:
        list_error_index.append(index)

In [5]:
np.savetxt("log_list_error.txt", list_error_index)

In [6]:
new_dataframe = masks.drop(list_error_index, axis=0)
print("new length =", len(new_dataframe))
print("diff length :", len(masks)-len(new_dataframe))
print("errors length :", len(list_error_index))

new length = 231722
diff length : 1
errors length : 1


In [7]:
new_dataframe.to_csv(ship_dir + "train_ship_segmentations_v2.csv")

## Removes duplicates

In [4]:
label_dir = os.path.join(ship_dir, 'train_ship_segmentations_v2_clean.csv')# Images for testing
dataframe = pd.read_csv(label_dir, engine="python") # Markers for ships

In [5]:
dataframe.head(5)

,Unnamed: 0,ImageId,EncodedPixels
0,0,00003e153.jpg,NaN
1,1,0001124c7.jpg,NaN
2,2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [18]:
np.dtype.float32

AttributeError: type object 'numpy.dtype' has no attribute 'float32'

In [23]:
np.array(dataframe['EncodedPixels'][2].split(" ")).astype(int)

array([264661,     17, 265429,     33, 266197,     33, 266965,     33,
       267733,     33, 268501,     33, 269269,     33, 270037,     33,
       270805,     33, 271573,     33, 272341,     33, 273109,     33,
       273877,     33, 274645,     33, 275413,     33, 276181,     33,
       276949,     33, 277716,     34, 278484,     34, 279252,     33,
       280020,     33, 280788,     33, 281556,     33, 282324,     33,
       283092,     33, 283860,     33, 284628,     33, 285396,     33,
       286164,     33, 286932,     33, 287700,     33, 288468,     33,
       289236,     33, 290004,     33, 290772,     33, 291540,     33,
       292308,     33, 293076,     33, 293844,     33, 294612,     33,
       295380,     33, 296148,     33, 296916,     33, 297684,     33,
       298452,     33, 299220,     33, 299988,     33, 300756,     33,
       301524,     33, 302292,     33, 303060,     33, 303827,     34,
       304595,     34, 305363,     33, 306131,     33, 306899,     33,
      

In [100]:
def encodedPixel2shape(x):
    x_str = str(x)
    if x_str != 'nan':
        shape = np.array(x_str.split(" ")).astype(int)[1:][::2]
    else:
        shape = 'nan'
    return str(shape).replace(",", " ").replace("[", "").replace("]", "")

In [32]:
dd

array([264661,     33, 266965,     33, 269269,     33, 271573,     33,
       273877,     33, 276181,     33, 278484,     33, 280788,     33,
       283092,     33, 285396,     33, 287700,     33, 290004,     33,
       292308,     33, 294612,     33, 296916,     33, 299220,     33,
       301524,     33, 303827,     34, 306131,     33, 308435,     33,
       310739,     33, 313043,     33, 315347,     33, 317651,     33,
       319955,     33, 322259,     33, 324563,     33, 326867,     33,
       329171,     34, 331474,     33, 333778,     33, 336082,     33,
       338386,     33, 340690,     33, 343003,      8])

In [101]:
#dataframe['EncodedPixels'][2].split(" ")[1:][::2]

In [102]:
encodedPixel2shape(dataframe['EncodedPixels'][2])

'17 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 34 34 33 33 33 33 33\n 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33\n 33 33 33 34 34 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33\n 33 33 33 33 33 33 33 33 33 33 33 33 33 34 34 33 33 33 33 33 33 33 33 33\n 33 33 33 33 33 33 24  8'

In [103]:
dataframe["shape"] = dataframe['EncodedPixels'].map(encodedPixel2shape)

In [104]:
dataframe.head(5)

,Unnamed: 0,ImageId,EncodedPixels,shape
0,0,00003e153.jpg,NaN,nan
1,1,0001124c7.jpg,NaN,nan
2,2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...,17 33 33 33 33 33 33 33 33 33 33 33 33 33 33 3...
3,3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...,1 4 5 8 10 10 9 10 10 9 9 10 10 9 9 ...
4,4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...,9 9 9 9 9 7 7 7 7 7 7 7 9 9 9 9 9 9


In [108]:
dataframe_noduplicates = dataframe.drop_duplicates("shape")

In [109]:
len(dataframe)

231722

In [110]:
len(dataframe_noduplicates)

17425

In [113]:
final_dataframe = dataframe_noduplicates[["ImageId", "EncodedPixels"]][1:]

In [114]:
final_dataframe.head(10)

,ImageId,EncodedPixels
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...
5,000194a2d.jpg,198320 10 199088 10 199856 10 200624 10 201392...
6,000194a2d.jpg,55683 1 56451 1 57219 1 57987 1 58755 1 59523 ...
7,000194a2d.jpg,254389 9 255157 17 255925 17 256693 17 257461 ...
9,00021ddc3.jpg,108287 1 109054 3 109821 4 110588 5 111356 5 1...
10,00021ddc3.jpg,101361 1 102128 3 102896 4 103663 6 104430 9 1...
11,00021ddc3.jpg,74441 3 75207 5 75975 5 76743 5 77511 5 78280 ...
12,00021ddc3.jpg,74444 4 75212 4 75980 4 76748 4 77517 3 78285 ...


In [ ]:
new_dataframe.to_csv(ship_dir + "train_ship_segmentations_v2_clean_nodup.csv", index=False)